## Machine Learning aided **Task prioritization**

Import the libraries and setup the **keywords**:

In [ ]:
import os
import tensorflow as tf

data = []
primes = []
keywords = ["add", "delete", "edit", "implement", "comment", "create", "update", "fix", "set", "use", "provide", "text",
            "write", "read", "dm", "call", "email", "tell", "prevent", "notify", "finish", "convert", "do", "buy",
            "deliver", "execute", "test", "sell", "open", "close", "stop", "continue", "cancel", "build", "merge",
            "replace", "swap", "fax", "mail", "important", "vital", "soon", "urgent", "emergency", "address", "issue",
            "problem", "discontinue", "take", "find"]

n = len(keywords)

Define the encoding function needed for extraction of the frequency domain of the keywords occurrences in the given text. This will be used for the input data of the neural network model later.

In [ ]:
def encode_text(text):
    text = text.lower()
    text = text.translate({ord(i): ' ' for i in '.,/!?[]()*&_-\\|#'})
    enc = []
    for i in range(n):
        enc.append(text.count(keywords[i]))
    return enc

Extract the training data and 'labels' (Priorities assigned to every task description by the user from 0 to 9 as 0 being the highest priority):

In [ ]:
train_data = []
train_prios = []

for f in os.listdir('data/train/tasks'):
    train_data.append(encode_text(open('data/train/tasks/' + f).read()))
    ls = [0 for i in range(10)]
    ls[int(open('data/train/rating/' + f).read())] = 1
    train_prios.append(ls)

Then configure our ML model, which consists of input layer denoting the frequency of every keyword [size: n] two hidden *dense* layers with ReLU (rectified linear unit) activations and final output layer with 10 nodes each denoting the probability of the **predicted priority** to be respectively 1 to 10, again 1 being the highest possible priority value.
We then run the model training for about 300 epochs since our dataset consists of barely 60 records:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=n),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_prios, epochs=300)

Now let's test how the model is reacting to new inputs from the test dataset.
To summarize the results of the neural network we calculate the expected value of the priority by adding the products of probability and sub-priority together. Then this is our final prediction:

In [ ]:
test_data = []

for f in os.listdir('data/test'):
    test_data.append(encode_text(open('data/test/' + f).read()))

for ls in model.predict(test_data):
    res = 0
    for i in range(10):
        res += (i+1) * ls[i]
    print(res)

The tests were the following:

As we see the second task gets way higher priority than the first one, mainly because of having some very important *keywords* like: **important** and **urgent**.

Thus we can conclude that despite our very small dataset of self written task descriptions and labeling the model we constructed is beginning to grasp some important concepts of our main idea. Of course to become way more accurate and consistant in the output, long time of data collecting, labeling and training is needed.

This is why the project is aimed to be self developing, which means that the model has to adapt to the given user input dynamically. For example the AI suggests you some ordering of your tasks in the Web App, but you think some other configuration would be better for you and reorder them in the online form. Well, congratulations! You helped us improve our ML model by contributing another training data.